<a href="https://colab.research.google.com/github/ajgrant6/Pokemon_LLM_Finetuner/blob/main/Pokemon_Translation_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install scikit-learn
!pip install transformers

In [25]:
from huggingface_hub import notebook_login

notebook_login()

# The Problem
Nintendo localizes the names of Pokemon, but a lot of translation systems fail to capture these localized names. I'd like to improve on these systems by finetuning them to understand these localized names.

# The Dataset

## Loading the Data

Here is the source of the dataset:

 https://www.pokecommunity.com/threads/international-list-of-names-in-csv.460446/

 https://docs.google.com/spreadsheets/d/1Eo6oWs4RA5M4c0r9M8FXJniOyhpmNmrnULabkP8kbL8/edit?usp=sharing&source=pokecommunity.com


In [26]:
from datasets import load_dataset

# https://www.pokecommunity.com/threads/international-list-of-names-in-csv.460446/
data = load_dataset("csv", data_files = "/content/PokemonNames.csv")

data

DatasetDict({
    train: Dataset({
        features: ['keys', 'en', 'ja', 'fr', 'de', 'es', 'it', 'ko', 'zh_HK', 'zh'],
        num_rows: 1025
    })
})

In [27]:
data["train"][0]["en"]

'Bulbasaur'

In [28]:
# Drop columns except for en and de

# Select the 'train' split
data_train = data['train']

# Remove all columns except 'en' and 'de'
columns_to_keep = ['en', 'de']
columns_to_remove = [col for col in data_train.column_names if col not in columns_to_keep]
filtered_train = data_train.remove_columns(columns_to_remove)

# Replace the original train split with the filtered one
data['train'] = filtered_train

data

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 1025
    })
})

# The Model

## Loading the Model

In [29]:
from transformers import pipeline

# We're gonna use the Helsinki English to German model
model_checkpoint = "Helsinki-NLP/opus-mt-en-de"

translator = pipeline("translation", model = model_checkpoint)
translator("Bulbasaur")

# The correct translation for "Bulbasaur" is "Bisasam"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Bulbasaur'}]

## Loading the Tokenizer

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

en_name = data["train"][0]["en"]
de_name = data["train"][0]["de"]

inputs = tokenizer(en_name, text_target=de_name)
inputs

{'input_ids': [17122, 5216, 111, 705, 0], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [3379, 111, 4457, 0]}

In [31]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])
# tokenizer.convert_ids_to_tokens(inputs["labels"])

['▁Bul', 'bas', 'a', 'ur', '</s>']

## Preprocess Function



In [32]:
def preprocess_function(examples):
    inputs = examples["en"]
    targets = examples["de"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

## Tokenization of the Dataset

# Finetuning
## Setting up the model

In [33]:
max_length = 16 # Pokemon names are usually short

tokenized_datasets = data.map(
    preprocess_function,
    batched=True,
    remove_columns = data["train"].column_names,
)

tokenized_datasets

Map:   0%|          | 0/1025 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1025
    })
})

In [34]:
# Print a few examples from the tokenized dataset to inspect their structure
for i in range(5):
    print(f"Example {i}: {tokenized_datasets['train'][i]}")

Example 0: {'input_ids': [17122, 5216, 111, 705, 0], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [3379, 111, 4457, 0]}
Example 1: {'input_ids': [38, 16117, 1423, 705, 0], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [3379, 1308, 877, 2069, 0]}
Example 2: {'input_ids': [21123, 111, 705, 0], 'attention_mask': [1, 1, 1, 1], 'labels': [3379, 111, 20321, 0]}
Example 3: {'input_ids': [26078, 8292, 0], 'attention_mask': [1, 1, 1], 'labels': [18846, 90, 7061, 0]}
Example 4: {'input_ids': [26078, 7459, 393, 0], 'attention_mask': [1, 1, 1, 1], 'labels': [37465, 18, 12757, 0]}


In [35]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Data Collator

In [36]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
# batch.keys()
# batch["decoder_input_ids"]
# batch["labels"]

for i in range(0, 3):
    print(tokenized_datasets["train"][i]["labels"])

[3379, 111, 4457, 0]
[3379, 1308, 877, 2069, 0]
[3379, 111, 20321, 0]


## Evaluation

In [37]:
import numpy as np
from datasets import load_metric
import evaluate

# Load the metric (e.g., BLEU)
metric = evaluate.load("exact_match")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute the metric
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"exact_match": result["exact_match"]}

## Fine-Tuning

In [38]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"pokemon-finetuned-opus-mt-en-de",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
print(tokenized_datasets["train"][0])

{'input_ids': [17122, 5216, 111, 705, 0], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [3379, 111, 4457, 0]}


In [40]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-665b9c8d-5fe68a43336ea8884c50f80a;ed49869d-9261-4077-b5a2-a260eac75895)

Invalid username or password.

In [ ]:
pre_tune_score = trainer.evaluate()
pre_tune_score

## Training

In [ ]:
trainer.train()

In [ ]:
post_tune_score = trainer.evaluate(max_length=max_length)
post_tune_score